In [44]:
import pandas as pd

# Define file paths for data files
# These paths point to CSV files containing relevant data for analysis.
file_path = r"C:/Users/Guillermo Leal/OneDrive - IPADE Business School/PythonCodes/Project2 Data/postings.csv"
file_path2 = r"C:/Users/Guillermo Leal/OneDrive - IPADE Business School/PythonCodes/Project2 Data/company_industries.csv"
file_path3 = r"C:/Users/Guillermo Leal/OneDrive - IPADE Business School/PythonCodes/Project2 Data/States.csv"
file_path4 = r"C:/Users/Guillermo Leal/OneDrive - IPADE Business School/PythonCodes/Project2 Data/Industries.csv"
file_path5 = r"C:/Users/Guillermo Leal/OneDrive - IPADE Business School/PythonCodes/Project2 Data/companies.csv"

# Load datasets into DataFrames
# Each file is read into a separate pandas DataFrame for analysis.
df = pd.read_csv(file_path)       # Job postings data
df2 = pd.read_csv(file_path2)     # Company industries data
df3 = pd.read_csv(file_path3)     # States mapping data
df4 = pd.read_csv(file_path4)     # Industries data
df5 = pd.read_csv(file_path5)     # Company details



In [45]:
# Calculate the 'Final salary' column
# Use the median salary if available; otherwise, calculate the average of max and min salary.
# If both are unavailable, assign None (missing value).
df['Final salary'] = df.apply(
    lambda row: row['med_salary'] if pd.notnull(row['med_salary']) else 
                ((row['max_salary'] + row['min_salary']) / 2 if pd.notnull(row['max_salary']) and pd.notnull(row['min_salary']) else None),
    axis=1
)

# Preview the first 10 rows of relevant columns to verify the calculations
print(df[['max_salary', 'med_salary', 'min_salary', 'Final salary']].head(10))

   max_salary  med_salary  min_salary  Final salary
0        20.0         NaN        17.0          18.5
1        50.0         NaN        30.0          40.0
2     65000.0         NaN     45000.0       55000.0
3    175000.0         NaN    140000.0      157500.0
4     80000.0         NaN     60000.0       70000.0
5        20.0         NaN        14.0          17.0
6    300000.0         NaN     60000.0      180000.0
7    120000.0         NaN     90000.0      105000.0
8         NaN         NaN         NaN           NaN
9         NaN       350.0         NaN         350.0


In [46]:
# Count the number of non-null values in the 'Final salary' column
# This step identifies how many rows have a valid (non-null) value for 'Final salary'.
non_null_count = df['Final salary'].notnull().sum()

# Print the total count of non-null values
print(f"Number of non-null values in 'Final salary': {non_null_count}")

# Calculate the percentage of non-null values relative to the total number of rows
# This provides an overview of data completeness for the 'Final salary' column.
percentage_non_null = (non_null_count / len(df)) * 100

# Print the percentage of non-null values, formatted to two decimal places
print(f"Percentage of non-null values in 'Final salary': {percentage_non_null:.2f}%")


Number of non-null values in 'Final salary': 36073
Percentage of non-null values in 'Final salary': 29.13%


In [47]:
# Define conversion factors to calculate hourly wages based on different pay periods
# The factors convert annual, monthly, biweekly, and weekly salaries to hourly wages.
conversion_factors = {
    "HOURLY": 1,                # No conversion needed for hourly pay
    "YEARLY": 1 / (40 * 52),    # Convert yearly salary assuming 40 hours/week and 52 weeks/year
    "MONTHLY": 1 / (40 * 4),    # Convert monthly salary assuming 40 hours/week and 4 weeks/month
    "BIWEEKLY": 1 / 80,         # Convert biweekly salary assuming 80 hours per 2 weeks
    "WEEKLY": 1 / 40            # Convert weekly salary assuming 40 hours/week
}

# Calculate the "Final hourly salary" column
# Multiply the 'Final salary' by the appropriate conversion factor based on 'pay_period'.
# If 'pay_period' is missing from conversion_factors or 'Final salary' is null, assign None.
df['Final hourly salary'] = df.apply(
    lambda row: row['Final salary'] * conversion_factors[row['pay_period']]
    if row['pay_period'] in conversion_factors and pd.notnull(row['Final salary']) else None,
    axis=1
)

# Preview the first 5 rows of relevant columns to verify the calculations
print(df[['Final salary', 'pay_period', 'Final hourly salary']].head())



   Final salary pay_period  Final hourly salary
0          18.5     HOURLY            18.500000
1          40.0     HOURLY            40.000000
2       55000.0     YEARLY            26.442308
3      157500.0     YEARLY            75.721154
4       70000.0     YEARLY            33.653846


In [48]:
# Create a mapping dictionary to associate company_id with the state
# This dictionary maps each company_id from df5 to its corresponding state.
state_mapping = df5.set_index('company_id')['state'].to_dict()

# Map the 'State' column in df using the mapping dictionary
# Use the mapping to assign the state based on the 'company_id' in df.
# If a company_id is not found in the mapping, assign "NA" as a default value.
df['State'] = df['company_id'].map(state_mapping).fillna('NA')

# Preview the first 5 rows of 'company_id' and 'State' columns to verify the mapping
print(df[['company_id', 'State']].head())


   company_id     State
0   2774458.0        NJ
1         NaN        NA
2  64896719.0      Ohio
3    766262.0  New York
4         NaN        NA


In [49]:
# Create a mapping dictionary to associate 'State 1' with 'State 2'
# This dictionary maps state names in 'State 1' from df3 to their corresponding values in 'State 2'.
state_mapping2 = df3.set_index('State 1')['State 2'].to_dict()

# Map the 'US_States_W/wage' column in df using the mapping dictionary
# Use the 'State' column in df to find corresponding 'State 2' values from the mapping.
# If no match is found, assign "NA" as the default value.
df['US_States_W/wage'] = df['State'].map(state_mapping2).fillna('NA')

# Preview the first 5 rows of 'State' and 'US_States_W/wage' columns to verify the mapping
print(df[['State', 'US_States_W/wage']].head())


      State US_States_W/wage
0        NJ       New Jersey
1        NA               NA
2      Ohio             Ohio
3  New York         New York
4        NA               NA


In [50]:
# Group the data by 'US_States_W/wage' and calculate the average hourly salary for each state
# This aggregates the 'Final hourly salary' column, computing the mean for each unique state in 'US_States_W/wage'.
average_salary_by_state = df.groupby('US_States_W/wage')['Final hourly salary'].mean().reset_index()

# Rename columns for better readability
# Rename the resulting columns to reflect their content: state names and the average hourly salary.
average_salary_by_state.columns = ['US_States_W/wage', 'Average Hourly Salary']

# Display the DataFrame with the calculated averages for verification
print(average_salary_by_state)


        US_States_W/wage  Average Hourly Salary
0                Alabama              36.247274
1                 Alaska              58.202768
2                Arizona              36.827052
3               Arkansas              43.069187
4             California             223.150683
5               Colorado              45.762985
6            Connecticut             226.642400
7               Delaware              39.835377
8   District of Columbia             426.434141
9                Florida             132.789394
10               Georgia              42.062129
11                Hawaii              30.438636
12                 Idaho            1504.236065
13              Illinois              40.662239
14               Indiana              42.151183
15                  Iowa              47.281956
16                Kansas              34.151592
17              Kentucky              37.790212
18             Louisiana              32.534875
19                 Maine              41

In [51]:

# Define bins and labels for grouping hourly salaries
# Bins group 'Final hourly salary' into ranges (e.g., 100-200, 200-300, ..., >1000) to detect potential anomalies.
bins = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, float('inf')]  # 'float('inf')' captures salaries >1000
labels = ['100-200', '200-300', '300-400', '400-500', '500-600', '600-700', '700-800', '800-900', '900-1000', '>1000']

# Categorize 'Final hourly salary' into defined bins
# Use pd.cut to assign each salary to a bin and label it for better readability.
df['Salary Range'] = pd.cut(df['Final hourly salary'], bins=bins, labels=labels, right=False)

# Count the number of entries in each salary range
# Aggregate the data to see how many salaries fall into each bin and sort for logical order.
salary_counts = df['Salary Range'].value_counts().sort_index()

# Convert the results to a DataFrame for visualization
# Reset index and rename columns to create a clean, readable table of salary ranges and their counts.
salary_counts_df = salary_counts.reset_index()
salary_counts_df.columns = ['Salary Range', 'Count']

# Display the salary distribution table
print("Hourly Salary Distribution:")
print(salary_counts_df)

# Investigate bins with extreme counts
# High counts in unexpected bins (e.g., >1000) may indicate outliers or typos in the data.



Hourly Salary Distribution:
  Salary Range  Count
0      100-200   1403
1      200-300     91
2      300-400     21
3      400-500      6
4      500-600      1
5      600-700      2
6      700-800      2
7      800-900      0
8     900-1000      3
9        >1000     31


In [52]:
# Define conversion factors to normalize salaries based on pay periods
# Factors convert yearly, monthly, biweekly, and weekly salaries into hourly wages.
conversion_factors = {
    "HOURLY": 1,                # No conversion needed for hourly pay
    "YEARLY": 1 / (40 * 52),    # Convert yearly salary assuming 40 hours/week, 52 weeks/year
    "MONTHLY": 1 / (40 * 4),    # Convert monthly salary assuming 40 hours/week, 4 weeks/month
    "BIWEEKLY": 1 / 80,         # Convert biweekly salary assuming 80 hours/2 weeks
    "WEEKLY": 1 / 40            # Convert weekly salary assuming 40 hours/week
}

# Step 1: Calculate 'Final hourly salary' based on pay period and final salary
# Use the appropriate conversion factor for the given 'pay_period' if both it and 'Final salary' are valid.
df['Final hourly salary'] = df.apply(
    lambda row: row['Final salary'] * conversion_factors[row['pay_period']]
    if row['pay_period'] in conversion_factors and pd.notnull(row['Final salary']) else None,
    axis=1
)

# Step 2: Replace values greater than $500/hour with NaN
# Salaries exceeding $500/hour are likely outliers or errors. Replace them with None to exclude from analysis.
df['Final hourly salary'] = df['Final hourly salary'].apply(
    lambda x: x if x is not None and x <= 500 else None
)

# Preview the first few rows of the updated DataFrame to verify calculations and outlier handling
print("Updated DataFrame with Final Hourly Salary (Outliers Removed):")
print(df[['Final salary', 'pay_period', 'Final hourly salary']].head())



Updated DataFrame with Final Hourly Salary (Outliers Removed):
   Final salary pay_period  Final hourly salary
0          18.5     HOURLY            18.500000
1          40.0     HOURLY            40.000000
2       55000.0     YEARLY            26.442308
3      157500.0     YEARLY            75.721154
4       70000.0     YEARLY            33.653846


In [53]:
# Group data by 'US_States_W/wage' and calculate the average hourly salary
# This step computes the mean 'Final hourly salary' for each unique state in the 'US_States_W/wage' column.
average_salary_by_state = df.groupby('US_States_W/wage')['Final hourly salary'].mean().reset_index()

# Rename columns for clarity
# Update the column names to reflect their content: the state and the calculated average hourly salary.
average_salary_by_state.columns = ['US_States_W/wage', 'Average Hourly Salary']

# Display the resulting DataFrame
# This shows the average hourly salary (with the updated salaries) by state for verification or further analysis.
print(average_salary_by_state)


        US_States_W/wage  Average Hourly Salary
0                Alabama              36.247274
1                 Alaska              58.202768
2                Arizona              36.827052
3               Arkansas              43.069187
4             California              55.075794
5               Colorado              45.762985
6            Connecticut              43.660427
7               Delaware              39.835377
8   District of Columbia              49.892678
9                Florida              44.862241
10               Georgia              42.062129
11                Hawaii              30.438636
12                 Idaho              36.781201
13              Illinois              40.662239
14               Indiana              42.151183
15                  Iowa              47.281956
16                Kansas              34.151592
17              Kentucky              37.790212
18             Louisiana              32.534875
19                 Maine              41

In [54]:
"""
This code interacts with the Bureau of Labor Statistics (BLS) API to fetch 2023 hourly wage data 
for each U.S. state. The goal is to calculate and compile the average wage by state, 
allowing us to compare this data with other sources such as LinkedIn.
"""

import requests
import pandas as pd

# Define your BLS API Key
API_KEY = "b638de5903e3465aab6ad81ace6d0ee1"

# Map state names to BLS series IDs for hourly wage data
state_series_ids = {
    "Alabama": "SMU01000000500000003",
    "Alaska": "SMU02000000500000003",
    "Arizona": "SMU04000000500000003",
    "Arkansas": "SMU05000000500000003",
    "California": "SMU06000000500000003",
    "Colorado": "SMU08000000500000003",
    "Connecticut": "SMU09000000500000003",
    "Delaware": "SMU10000000500000003",
    "District of Columbia": "SMU11000000500000003",
    "Florida": "SMU12000000500000003",
    "Georgia": "SMU13000000500000003",
    "Hawaii": "SMU15000000500000003",
    "Idaho": "SMU16000000500000003",
    "Illinois": "SMU17000000500000003",
    "Indiana": "SMU18000000500000003",
    "Iowa": "SMU19000000500000003",
    "Kansas": "SMU20000000500000003",
    "Kentucky": "SMU21000000500000003",
    "Louisiana": "SMU22000000500000003",
    "Maine": "SMU23000000500000003",
    "Maryland": "SMU24000000500000003",
    "Massachusetts": "SMU25000000500000003",
    "Michigan": "SMU26000000500000003",
    "Minnesota": "SMU27000000500000003",
    "Mississippi": "SMU28000000500000003",
    "Missouri": "SMU29000000500000003",
    "Montana": "SMU30000000500000003",
    "Nebraska": "SMU31000000500000003",
    "Nevada": "SMU32000000500000003",
    "New Hampshire": "SMU33000000500000003",
    "New Jersey": "SMU34000000500000003",
    "New Mexico": "SMU35000000500000003",
    "New York": "SMU36000000500000003",
    "North Carolina": "SMU37000000500000003",
    "North Dakota": "SMU38000000500000003",
    "Ohio": "SMU39000000500000003",
    "Oklahoma": "SMU40000000500000003",
    "Oregon": "SMU41000000500000003",
    "Pennsylvania": "SMU42000000500000003",
    "Rhode Island": "SMU44000000500000003",
    "South Carolina": "SMU45000000500000003",
    "South Dakota": "SMU46000000500000003",
    "Tennessee": "SMU47000000500000003",
    "Texas": "SMU48000000500000003",
    "Utah": "SMU49000000500000003",
    "Vermont": "SMU50000000500000003",
    "Virginia": "SMU51000000500000003",
    "Washington": "SMU53000000500000003",
    "West Virginia": "SMU54000000500000003",
    "Wisconsin": "SMU55000000500000003",
    "Wyoming": "SMU56000000500000003"
}

# Fetch wage data for a specific state in 2023
def fetch_wages_2023(state_name, series_id):
    params = {
        "seriesid": [series_id],
        "startyear": "2023",
        "endyear": "2023",
        "registrationkey": API_KEY
    }
    response = requests.post("https://api.bls.gov/publicAPI/v2/timeseries/data/", json=params)
    if response.status_code == 200:
        data = response.json()
        if 'Results' in data and 'series' in data['Results']:
            # Extract hourly wage for 2023
            for entry in data['Results']['series'][0]['data']:
                if entry["year"] == "2023":
                    return float(entry["value"])
    return None

# Fetch wages for all states
wages_2023 = {state: fetch_wages_2023(state, series_id) for state, series_id in state_series_ids.items()}

# Convert wage data into a DataFrame
df_wages_2023 = pd.DataFrame(list(wages_2023.items()), columns=["State", "Hourly Wage 2023"])

# Display the DataFrame with average hourly wages by state
print("Average Hourly Wages by State (2023, BLS Data):")
print(df_wages_2023)


Average Hourly Wages by State (2023, BLS Data):
                   State  Hourly Wage 2023
0                Alabama             30.09
1                 Alaska             35.52
2                Arizona             31.94
3               Arkansas             27.92
4             California             38.42
5               Colorado             36.94
6            Connecticut             36.93
7               Delaware             31.50
8   District of Columbia             51.06
9                Florida             31.92
10               Georgia             30.91
11                Hawaii             36.17
12                 Idaho             31.08
13              Illinois             33.65
14               Indiana             29.81
15                  Iowa             28.98
16                Kansas             29.58
17              Kentucky             28.60
18             Louisiana             27.89
19                 Maine             31.16
20              Maryland             34.84
21    

In [55]:
# Merge LinkedIn and BLS DataFrames on the "State" column
# Combine LinkedIn and BLS hourly wage data for comparison, aligning on state names.
comparison_df = pd.merge(df_wages_2023, average_salary_by_state, left_on="State", right_on="US_States_W/wage")

# Calculate the absolute and percentage differences in wages
# 'Difference (Units)' computes LinkedIn - BLS for hourly wages.
comparison_df["Difference (Units)"] = comparison_df["Average Hourly Salary"] - comparison_df["Hourly Wage 2023"]
# 'Difference (%)' calculates the percentage difference relative to BLS wages.
comparison_df["Difference (%)"] = (comparison_df["Difference (Units)"] / comparison_df["Hourly Wage 2023"]) * 100

# Format numeric columns for readability
# Convert all monetary and percentage values into a more readable format.
comparison_df["LinkedIn"] = comparison_df["Average Hourly Salary"].apply(lambda x: f"{x:,.2f}")
comparison_df["BLS"] = comparison_df["Hourly Wage 2023"].apply(lambda x: f"{x:,.2f}")
comparison_df["Difference (Units)"] = comparison_df["Difference (Units)"].apply(lambda x: f"{x:,.2f}")
comparison_df["Difference (%)"] = comparison_df["Difference (%)"].apply(lambda x: f"{x:,.2f}%")

# Select and display relevant columns
# Keep only columns relevant for comparison and present the final DataFrame.
comparison_df = comparison_df[["State", "LinkedIn", "BLS", "Difference (Units)", "Difference (%)"]]

# Display the comparison results
print("Comparison of Average Hourly Wages by State (LinkedIn vs. BLS):")
comparison_df


Comparison of Average Hourly Wages by State (LinkedIn vs. BLS):


,State,LinkedIn,BLS,Difference (Units),Difference (%)
0,Alabama,36.25,30.09,6.16,20.46%
1,Alaska,58.20,35.52,22.68,63.86%
2,Arizona,36.83,31.94,4.89,15.30%
3,Arkansas,43.07,27.92,15.15,54.26%
4,California,55.08,38.42,16.66,43.35%
5,Colorado,45.76,36.94,8.82,23.88%
6,Connecticut,43.66,36.93,6.73,18.22%
7,Delaware,39.84,31.50,8.34,26.46%
8,District of Columbia,49.89,51.06,-1.17,-2.29%
9,Florida,44.86,31.92,12.94,40.55%


In [56]:
# Calculate national averages for LinkedIn and BLS hourly wages
# Compute the mean hourly wage across all states for both LinkedIn and BLS datasets.
national_linkedin_avg = comparison_df["LinkedIn"].astype(float).mean()  # Convert formatted strings to floats
national_bls_avg = comparison_df["BLS"].astype(float).mean()

# Calculate absolute and percentage differences
# 'Difference (Units)' is the difference between LinkedIn and BLS averages.
difference_units = national_linkedin_avg - national_bls_avg
# 'Difference (%)' calculates the percentage difference relative to BLS wages.
difference_percentage = (difference_units / national_bls_avg) * 100

# Display the national-level comparison results
print(f"National LinkedIn Average Hourly Salary: ${national_linkedin_avg:,.2f}")
print(f"National BLS Average Hourly Salary: ${national_bls_avg:,.2f}")
print(f"Difference (Units): ${difference_units:,.2f}")
print(f"Difference (%): {difference_percentage:,.2f}%")


National LinkedIn Average Hourly Salary: $40.80
National BLS Average Hourly Salary: $32.75
Difference (Units): $8.05
Difference (%): 24.58%
